In [20]:
import os
from pathlib import Path
import fiona
import geopandas as gpd


In [21]:
gdb = "/import/beegfs/CMIP6/jdpaul3/hydroviz_data/gis/GeospatialFabric_National.gdb"
seg_shp = "/import/beegfs/CMIP6/jdpaul3/hydroviz_data/gis/Segments_subset.shp"

fiona.listlayers(gdb)

['POIs',
 'one',
 'nhdflowline_en',
 'nhdflowline',
 'regionOutletDA',
 'nhruNationalIdentifier',
 'nsegmentNationalIdentifier']

In [22]:
poi_gdf = gpd.read_file(gdb, layer='POIs', encoding='utf-8')
gf_seg_gdf = gpd.read_file(gdb, layer='nsegmentNationalIdentifier', encoding='utf-8')

In [23]:
# subset geospatial fabric segments
gf_seg = gf_seg_gdf[['seg_id_nat', 'POI_ID']]
gf_seg['POI_ID'] = gf_seg['POI_ID'].astype(int)

/tmp/ipykernel_1176096/1751004690.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gf_seg['POI_ID'] = gf_seg['POI_ID'].astype(int)


In [24]:
# prep POI gdf
poi = poi_gdf[['COMID', 'GNIS_NAME', 'poi_gage_id']]
poi['COMID'] = poi['COMID'].astype(int)

/tmp/ipykernel_1176096/2210299984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi['COMID'] = poi['COMID'].astype(int)


In [25]:
# join POI to geospatial fabricsegments
gf_seg = gf_seg.join(poi.set_index('COMID'), on='POI_ID')

In [26]:
# read in hydroviz segments
hv_seg_gdf = gpd.read_file(seg_shp, encoding='utf-8')

In [27]:
# join hydroviz segments to geospatial fabric segments with POI info
seg_gdf = hv_seg_gdf.join(gf_seg.set_index('seg_id_nat'), on='seg_id_nat')

# add "USGS-" prefix to 'poi_gage_id'
seg_gdf['GAUGE_ID'] = seg_gdf['poi_gage_id'].apply(lambda x: f"USGS-{x}")

# replace "USGS-0" with "NA"
seg_gdf['GAUGE_ID'] = seg_gdf['GAUGE_ID'].replace("USGS-0", "NA")

# drop unnecessary columns
seg_gdf = seg_gdf[['seg_id_nat', 'GNIS_NAME', 'GAUGE_ID', 'geometry']]

seg_gdf


,seg_id_nat,GNIS_NAME,GAUGE_ID,geometry
0,1,West Branch Mattawamkeag River,NA,"LINESTRING (2101948.624 2876678.641, 2101941.3..."
1,2,Baskahegan Stream,NA,"LINESTRING (2167789.031 2829021.852, 2167729.9..."
2,3,Mattawamkeag River,NA,"LINESTRING (2131936.492 2865675.020, 2131955.7..."
3,4,Mattawamkeag River,NA,"LINESTRING (2151719.943 2849594.051, 2151812.0..."
4,5,Baskahegan Stream,NA,"LINESTRING (2155981.103 2842240.715, 2155894.2..."
...,...,...,...,...
56455,56456,,NA,"LINESTRING (-1943922.030 1664269.470, -1943834..."
56456,56457,,NA,"LINESTRING (-1939576.427 1684173.491, -1939582..."
56457,56458,Owens River,USGS-10277500,"LINESTRING (-1941082.934 1778819.137, -1941078..."
56458,56459,,NA,"LINESTRING (-2283644.191 2427521.123, -2283652..."


In [28]:
len(seg_gdf["GAUGE_ID"].unique())

8273

In [29]:
output_dir = "/import/beegfs/CMIP6/jdpaul3/hydroviz_data/export"
seg_gdf.to_file(os.path.join(output_dir, 'conus_segments.shp'))